In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [50]:
batting_cols = ['MatchPlayed','InningsBatted','NotOuts','RunsScored','HighestInnScore','100s','50s','4s','6s','BattingAVG','BattingS/R','CatchesTaken','StumpingsMade','Ducks','R/O']
bowling_cols = ['InningsBowled','Overs','Maidens','RunsConceded','Wickets','Best','3s','5s','BowlingAVG','EconomyRate','S/R','Mtc']

In [51]:
def fetch_Player(url,player_dict):
    try:
        player_html_text = requests.get('https://sports.ndtv.com'+url).text
        player_soup = BeautifulSoup(player_html_text, 'lxml')
        bio = player_soup.find('div', class_='plyr-bio_wrp').tbody
        bio_rows = bio.find_all('tr', class_='plyr-bio_tr')
        for row in bio_rows:
            player_dict[row.select('td.plyr-bio_tdLT')[0].text] = row.select('td.plyr-bio_tdR')[0].text
        batbowl = player_soup.find_all('div', class_='Plyr-Tb_tbl-cnt')
        if len(batbowl) != 2:
            print(url,len(batbowl))
        else:
            batting = batbowl[0]
            stats = batting.find('span', text='IPL')
            if stats:
                batting_stats = stats.parent.parent.parent.parent
                x = batting_stats.find_all('td', class_='Plyr-Tb_tbl-td Plyr-Tb_tx-ctr')
                for i in range(len(x)):
                    player_dict[batting_cols[i]] = x[i].text
                    player_dict['Sport'] = 'IPL'
            else:
                for x in batting_cols:
                    player_dict[x] = ''
                    player_dict['Sport'] = ''
            
            bowling = batbowl[1]
            stats = bowling.find('span', text='IPL')
            if stats:
                bowling_stats = stats.parent.parent.parent.parent
                x = bowling_stats.find_all('td', class_='Plyr-Tb_tbl-td Plyr-Tb_tx-ctr')
                for i in range(len(x)):
                    player_dict[bowling_cols[i]] = x[i].text
                    player_dict['Sport'] = 'IPL'
            else:
                for x in bowling_cols:
                    player_dict[x] = ''
                    player_dict['Sport'] = ''
    except:
        print(url)

In [52]:
html_text = requests.get('https://sports.ndtv.com/ipl-2022/auction/teamsquad#squad_csk').text
soup = BeautifulSoup(html_text, 'lxml')
squads = soup.find_all('div', class_='AUipl_tbl-cnt squaddm')
squad_dict = {'squad_pbks':'PBKS'}
player_list = [] 
for squad in squads:
    team_name = squad['data-attr'].split('_')[1].upper()
    # print(team_name)
    # print('--------------------------------')
    players = squad.tbody.find_all('tr', class_='AUipl_tbl-tr')

    for player in players:
        player_dict = {}
        # Player Name
        name = player.select('td:nth-child(2) > span.AUipl_tbl-nm.AUipl_tbl-PlIc > a > span:nth-child(2)')
        player_dict['Name'] = name[0].text

        player_dict['Team'] = team_name
        
        url = player.select('td:nth-child(2) > span.AUipl_tbl-nm.AUipl_tbl-PlIc > a')
        player_dict['Url'] = 'https://sports.ndtv.com' + url[0]['href']
        
        type = player.select('td:nth-child(3) > div')
        player_dict['Type'] = type[0].text
        
        value = player.select('td:nth-child(4) > span')
        player_dict['ValueinCR'] = value[0].text

        fetch_Player(url[0]['href'],player_dict)
        player_list.append(player_dict)

/cricket/players/43735-pramod-bhanuka-bandara-rajapaksa-playerprofile 0
/cricket/players/45414-benny-alexander-cameron-howell-playerprofile 0
/cricket/players/63978-ravikumar-samarth-playerprofile 0
/cricket/players/1541-rassie-van-der-dussen-playerprofile
/cricket/players/64116-chama-v-milind-playerprofile
/cricket/players/1603-dwaine-pretorius-playerprofile
/cricket/players/27641-sheldon-philip-jackson-playerprofile
/cricket/players/64316-chamika-karunaratne-playerprofile 0
/cricket/players/1682-evin-lewis-playerprofile
/cricket/players/64845-rahul-tewatia-playerprofile


In [53]:
# player_list

In [54]:
df = pd.DataFrame(player_list)
df

,Name,Team,Url,Type,ValueinCR,Full Name,Born,Age,National Side,Batting Style,...,Maidens,RunsConceded,Wickets,Best,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
0,Mayank Agarwal,PBKS,https://sports.ndtv.com/cricket/players/1430-m...,Batsman,12.00,Mayank Anurag Agarwal,"February 16, 1991 Bangalore, Karnataka","31 Years, 0 Months, 28 Days",India,Right Handed,...,,,,,,,,,,
1,Liam Livingstone,PBKS,https://sports.ndtv.com/cricket/players/64363-...,All-Rounder,11.50,Liam Stephen Livingstone,"August 4, 1993 Barrow-in-Furness, Cumberland","28 Years, 7 Months, 11 Days",England,Right Handed,...,0,13,0,0/13 v MI,0,0,-,13.00,-,1
2,Kagiso Rabada,PBKS,https://sports.ndtv.com/cricket/players/64042-...,Bowler,9.25,Kagiso Rabada,"May 25, 1995 Johannesburg","26 Years, 9 Months, 22 Days",South Africa,Left Handed,...,2,1560,76,4/21 v RCB,4,0,20.52,8.21,15.00,50
3,Shahrukh Khan,PBKS,https://sports.ndtv.com/cricket/players/113433...,All-Rounder,9.00,Masood Shahrukh Khan,"May 27, 1995 Chennai, Tamil Nadu","26 Years, 9 Months, 20 Days",India,Right Handed,...,,,,,,,,,,
4,Shikhar Dhawan,PBKS,https://sports.ndtv.com/cricket/players/737-sh...,Batsman,8.25,Shikhar Dhawan,"December 5, 1985 Delhi","36 Years, 3 Months, 10 Days",India,Left Handed,...,0,66,4,1/7 v DC,0,0,16.50,8.25,12.00,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Gurkeerat Singh Mann,GT,https://sports.ndtv.com/cricket/players/1661-g...,All-Rounder,0.50,Gurkeerat Rupinder Singh,"June 29, 1990 Muktsar, Punjab","31 Years, 8 Months, 16 Days",India,Right Handed,...,0,97,5,2/15 v RPS,0,0,19.40,7.46,15.60,6
233,Noor Ahmad,GT,https://sports.ndtv.com/cricket/players/112545...,Bowler,0.30,Noor Ahmad Lakanwal,"January 3, 2005","17 Years, 2 Months, 12 Days",Afghanistan,Right Handed,...,,,,,,,,,,
234,Darshan Nalkande,GT,https://sports.ndtv.com/cricket/players/110589...,All-Rounder,0.20,Darshan Girish Nalkande,"October 4, 1998 Wardha, Maharashtra","23 Years, 5 Months, 11 Days",India,Right Handed,...,,,,,,,,,,
235,Sai Sudharsan,GT,https://sports.ndtv.com/cricket/players/115302...,All-Rounder,0.20,Bhardwaj Sai Sudharsan,"October 15, 2001","20 Years, 5 Months, 0 Days",India,Left Handed,...,,,,,,,,,,


In [55]:
df.isnull().sum().sum()

322

In [56]:
df.replace(to_replace =["-"], 
                            value ="",inplace = True)

In [57]:
df.to_csv('IPL_Data.csv',index=False)

In [58]:
df[df.isna().any(axis=1)]

,Name,Team,Url,Type,ValueinCR,Full Name,Born,Age,National Side,Batting Style,...,Maidens,RunsConceded,Wickets,Best,3s,5s,BowlingAVG,EconomyRate,S/R,Mtc
15,Bhanuka Rajapaksa,PBKS,https://sports.ndtv.com/cricket/players/43735-...,Batsman,0.50,Pramod Bhanuka Bandara Rajapaksa,"October 24, 1991","30 Years, 4 Months, 23 Days",Sri Lanka,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Benny Howell,PBKS,https://sports.ndtv.com/cricket/players/45414-...,All-Rounder,0.40,Benny Alexander Cameron Howell,"October 05, 1988","33 Years, 5 Months, 10 Days",England,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Ravikumar Samarth,SRH,https://sports.ndtv.com/cricket/players/63978-...,Batsman,0.20,Ravikumar Samarth,"January 22, 1993","29 Years, 1 Months, 21 Days",India,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Rassie van der Dussen,RR,https://sports.ndtv.com/cricket/players/1541-r...,Batsman,1.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Chama Milind,RCB,https://sports.ndtv.com/cricket/players/64116-...,Bowler,0.25,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,Dwaine Pretorius,CSK,https://sports.ndtv.com/cricket/players/1603-d...,All-Rounder,0.50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,Alex Hales,KKR,https://sports.ndtv.com/cricket/players/1562-a...,Batsman,1.50,Alexander Daniel Hales,"January 3, 1989 Hillingdon, Middlesex","33 Years, 2 Months, 12 Days",England,Right Handed,...,,,,,,,,,,
158,Sheldon Jackson,KKR,https://sports.ndtv.com/cricket/players/27641-...,Wicket-Keeper,0.60,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,Chamika Karunaratne,KKR,https://sports.ndtv.com/cricket/players/64316-...,All-Rounder,0.50,Chamika Karunaratne,"May 29, 1996","25 Years, 9 Months, 17 Days",Sri Lanka,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,Evin Lewis,LSG,https://sports.ndtv.com/cricket/players/1682-e...,Batsman,2.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
